In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


**Conjuntos** del modelo de optimización

| **Término** | **Descripción**                |
|-------------|--------------------------------|
| I           | Conjunto de productores        |
| J           | Conjunto de centros de acopio  |
| K           | Conjunto de clientes           |
| P           | Conjunto de productos          |
| T           | Conjunto de periodos de tiempo |

In [2]:
# Variables
I = 'Productores'
J = 'CAcopios'
K = 'Clientes'
P = 'Productos'
T = 'Tiempo'

In [3]:
csv_base = '../CSV/'

routes = {
    'cap_almacenamiento_ca' : 'CapAlmacenamientoCA.csv',
    'cap_productor' : 'CapProductor.csv',
    'c_produccion' : 'CostoProduccion.csv',
    'c_fijo_acopio': 'CostoFijoAcopio.csv',
    'c_var_acopio' : 'CostoVarAcopio.csv',
    'c_inv_acopio' : 'CostoInvAcopio.csv',
    'c_trans_prod_acopio' : 'CostoTransProdAcopio.csv',
    'c_trans_acopio_clie' : 'CostoTransAcopioClie.csv',
    'demanda_clie' : 'DemandaClie.csv'
}

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

# pd.read_csv(r'' + csv_base + routes.get('...'), delimiter=csv_config.get('delimiter'), decimal=csv_config.get('decimal'))

In [4]:
P_df = pd.read_csv(r'' + csv_base + P + '.csv')
P_df.head()

,Productos
0,Queso duro
1,Queso blando


## Escenarios de prueba de demandas por centro de acopio
Escenario 1: Un centro de acopio sobrepasa la demanda del cliente
- El modelo puede asignar dicho centro de acopio para suplir con la demanda
- El centro de acopio debe proveer esa demanda

Escenario 2: Ningún centro de acopio cumple con toda la demanda del cliente
- El modelo debe dar una solución de varios centros de acopio
- La suma de la demanda por cada centro de acopio debe ser igual a la demanda

Escenario 3: Un centro de acopio tiene en stock exactamente la cantidad de la demanda de un cliente.
- El modelo puede tomar el centro de acopio que tiene en stock exactamente la demanda a cumplir

### Generando demandas por cliente

In [5]:
import random

K_df = pd.read_csv(r'' + csv_base + K + '.csv')
clients = K_df[K].drop_duplicates()

X = []

for client in clients:
  X.append(10 * random.randint(0, 100))

clients = pd.DataFrame(clients)

clients['Demanda'] = pd.DataFrame(X)
clients

,Clientes,Demanda
0,Santa Marta,910
1,Riohacha,270
2,Monteria,260
3,Barranquilla,270
4,Bogotá,520
5,Bucaramanga,930
6,Cartagena,260
7,Cali,870
8,Cúcuta,350
9,Medellín,820


In [6]:
# Tomando una demanda
clients.loc[0]

Clientes    Santa Marta
Demanda             910
Name: 0, dtype: object

### Generando capacidades para cada centro de acopio

In [7]:
J_df = pd.read_csv(r'' + csv_base + J + '.csv')
c_acopios = J_df['Cacopio']
X = []

for c_acopio in c_acopios:
  X.append(10 * random.randint(0, 100))

c_acopios = pd.DataFrame(c_acopios)
c_acopios['Capacidad'] = pd.DataFrame(X)
c_acopios

,Cacopio,Capacidad
0,San Juan del Cesar,920
1,Riohacha,90
2,Maicao,210
3,Guamal,860
4,Pivijay,800
5,Nueva granada,590
6,Monteria,340
7,Chinu,90
8,Planeta rica,170


### func. obj

$$
Min(F) = CProduccion_t + COperacion_t + CInventario_t + CTransporte_t
$$

#### Costos de producción
$$
  CProduccion_t = \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} CostoProduccion_{pit} PA_{pijt} \qquad \forall_t \in T
$$

In [8]:
cp_df = pd.read_csv(r'' + csv_base + routes.get('c_produccion'))

cp_sum = cp_df.groupby('Productores')['CostoProduccion'].agg('sum')
cp_sum

Productores
ALBERTO URIANO                     454118
ALEXANDER VASQUEZ ROMERO           451138
ALFONSO GUERRA ARROYO              464352
ALVARO DE JESUS GUERRA AMAYA       454118
ANA HELENA BETANCOURT SIERRA       464352
                                    ...  
VICTOR SALCEDO CASTRO              464352
WILSON ALFREDO IBARRA DELUQUE      451138
WILSON ORJUELA PATIÑO              454118
YAJAIRA CATALINA AGUIRRE FLOREZ    468276
YOLMIS AMAYA ORTIZ                 454118
Name: CostoProduccion, Length: 103, dtype: int64

#### Costos de Operación
$$
  COperacion_t = CFijos_t + CVariables_t
$$

In [9]:
cfa_df = pd.read_csv(r'' + csv_base + routes.get('c_fijo_acopio'))

cfa_sum = cfa_df.groupby('CAcopios')['CostoFijoAcopio'].agg('sum')
cfa_sum

CAcopios
Chinu                 58440000
Guamal                51600000
Maicao                51960000
Monteria              57588000
Nueva granada         51960000
Pivijay               53400000
Planeta rica          57000000
Riohacha              53400000
San Juan del Cesar    51600000
Name: CostoFijoAcopio, dtype: int64

In [10]:
cva_df = pd.read_csv(r'' + csv_base + routes.get('c_var_acopio'))
cva_sum = cva_df.groupby('CAcopios')['CostoVarAcopio'].agg('sum')
cva_sum

CAcopios
Chinu                 21000
Guamal                18480
Maicao                18840
Monteria              22080
Nueva granada         18840
Pivijay               19440
Planeta rica          20400
Riohacha              19440
San Juan del Cesar    18480
Name: CostoVarAcopio, dtype: int64

#### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [11]:
cia_df = pd.read_csv(r'' + csv_base + routes.get('c_inv_acopio'))

cia_sum = cia_df.groupby('CAcopios')['CostoInvAcopio'].agg('sum')
cia_sum

CAcopios
Chinu                 13200
Guamal                10800
Maicao                13200
Monteria              13200
Nueva granada         13200
Pivijay               12000
Planeta rica          12000
Riohacha              12000
San Juan del Cesar    10800
Name: CostoInvAcopio, dtype: int64

#### Costos de Transporte
$$
  CTransporte_t = \sum_{j \in J} \sum_{i \in I} \sum_{p \in P} CostoTransAcopio_{pijt} PA_{pijt} + \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} \qquad \forall_t \in T
$$

In [12]:
ctpa_df = pd.read_csv(r'' + csv_base + routes.get('c_trans_prod_acopio'))

ctpa_sum = ctpa_df.groupby('CAcopios')['CostoTransProdAcopio'].agg('sum')
ctpa_sum

CAcopios
Chinu                 140256.00
Guamal                148307.64
Maicao                265420.80
Monteria              124776.00
Nueva granada         129672.00
Pivijay               194010.00
Planeta rica          122592.00
Riohacha              269722.80
San Juan del Cesar    267592.32
Name: CostoTransProdAcopio, dtype: float64

In [13]:
ctac_df = pd.read_csv(r'' + csv_base + routes.get('c_trans_acopio_clie'))

ctac_sum = ctac_df.groupby('CAcopios')['CostoTransAcopioClie'].agg('sum')
ctac_sum

CAcopios
Chinu                 213264.0
Guamal                135416.4
Maicao                246598.8
Monteria              225888.0
Nueva granada         283290.0
Pivijay               250152.0
Planeta rica          231072.0
Riohacha              257730.0
San Juan del Cesar    269108.4
Name: CostoTransAcopioClie, dtype: float64

### scipy

In [14]:
from scipy.optimize import dual_annealing

def f(x):
  x = x * sum(clients['Demanda'].values.tolist())
  return x

bounds = [(-2000, 2000)]
result = dual_annealing(f, bounds=bounds)
print(result)

 message: ['Maximum number of iteration reached']
 success: True
  status: 0
     fun: -12120000.0
       x: [-2.000e+03]
     nit: 1000
    nfev: 2005
    njev: 2
    nhev: 0


In [15]:
dc_df = pd.read_csv(r'' + csv_base + routes.get('demanda_clie'))
res = dc_df.groupby('Clientes')['DemandaClie'].agg('sum')

# Capacidades

In [16]:
cpr_df = pd.read_csv(r'' + csv_base + routes.get('cap_productor'))
cpr_df.head()

,Ciudad,Productos,Productores,Tiempo,CapProductor
0,Cordoba,Queso duro,FRANCISCO PEREZ,1,33900
1,Cordoba,Queso duro,FRANCISCO PEREZ,2,33900
2,Cordoba,Queso duro,FRANCISCO PEREZ,3,33900
3,Cordoba,Queso duro,FRANCISCO PEREZ,4,33900
4,Cordoba,Queso duro,FRANCISCO PEREZ,5,33900


In [17]:
cac_df = pd.read_csv(r'' + csv_base + routes.get('cap_almacenamiento_ca'))
cac_df.head()

,Ciudad,Productos,CAcopios,Tiempo,CapAlmacenamientoCA
0,La Guajira,Queso duro,San Juan del Cesar,1,30000
1,La Guajira,Queso duro,San Juan del Cesar,2,30000
2,La Guajira,Queso duro,San Juan del Cesar,3,30000
3,La Guajira,Queso duro,San Juan del Cesar,4,30000
4,La Guajira,Queso duro,San Juan del Cesar,5,30000
